In [1]:
import I2MC
import pandas as pd
import numpy as np

In [2]:
opt = dict()
# General variables for eye-tracking data
# maximum value of horizontal resolution in pixels
opt['xres'] = 1920.0
opt['yres'] = 1080.0  # maximum value of vertical resolution in pixels
# missing value for horizontal position in eye-tracking data (example data uses -xres). used throughout
# internal_helpers as signal for data loss
opt['missingx'] = -opt['xres']
# missing value for vertical position in eye-tracking data (example data uses -yres). used throughout
# internal_helpers as signal for data loss
opt['missingy'] = -opt['yres']
# sampling frequency of data (check that this value matches with values actually obtained from measurement!)
opt['freq'] = 250.0

# Variables for the calculation of visual angle
# These values are used to calculate noise measures (RMS and BCEA) of
# fixations. The may be left as is, but don't use the noise measures then.
# If either or both are empty, the noise measures are provided in pixels
# instead of degrees.
# screen size in cm
opt['scrSz'] = [53.1, 29.8]
# distance to screen in cm.
opt['disttoscreen'] = 65.0

# STEFFEN INTERPOLATION
# max duration (s) of missing values for interpolation to occur
opt['windowtimeInterp'] = 0.1
# amount of data (number of samples) at edges needed for interpolation
opt['edgeSampInterp'] = 2
# maximum displacement during missing for interpolation to be possible
opt['maxdisp'] = opt['xres'] * 0.2 * np.sqrt(2)

# # K-MEANS CLUSTERING
# time window (s) over which to calculate 2-means clustering (choose value so that max. 1 saccade can occur)
opt['windowtime'] = 0.2
# time window shift (s) for each iteration. Use zero for sample by sample processing
opt['steptime'] = 0.02
# maximum number of errors allowed in k-means clustering procedure before proceeding to next file
opt['maxerrors'] = 100
opt['downsamples'] = [2.0, 5.0, 10.0]
# use chebychev filter when down sampling? 1: yes, 0: no. requires signal processing toolbox. is what matlab's
# down sampling internal_helpers do, but could cause trouble (ringing) with the hard edges in eye-movement data
opt['downsampFilter'] = 0

# # FIXATION DETERMINATION
# number of standard deviations above mean k-means weights will be used as fixation cutoff
opt['cutoffstd'] = 2.0
# number of MAD away from median fixation duration. Will be used to walk forward at fixation starts and backward at
# fixation ends to refine their placement and stop algorithm from eating into saccades
opt['onoffsetThresh'] = 3.0
# maximum Euclidean distance in pixels between fixations for merging
opt['maxMergeDist'] = 30.0
# maximum time in ms between fixations for merging
opt['maxMergeTime'] = 30.0
# minimum fixation duration after merging, fixations with shorter duration are removed from output
opt['minFixDur'] = 40.0

In [11]:
df_behavioral = pd.read_csv("./data/filteredData/filtered_data.csv")
#iterate through each row
for index, row in df_behavioral.iterrows():
    print("{}-{}: Fixation Calculation".format(row["Participant"], row["Algorithm"]))
    df_eyetracking = pd.read_csv(row["Eyetracking"])
    df_eyetracking["time"] = df_eyetracking["time"].astype(float)
    df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"].iloc[0]
    # drop col
    df_eyetracking = df_eyetracking.drop(columns=["l_gaze_point_in_user_coordinate_system_x",
                                                  "l_gaze_point_in_user_coordinate_system_y",
                                                  "l_gaze_point_in_user_coordinate_system_z",
                                                  "r_gaze_point_in_user_coordinate_system_x",
                                                  "r_gaze_point_in_user_coordinate_system_y",
                                                  "r_gaze_point_in_user_coordinate_system_z",
                                                  "l_gaze_origin_in_user_coordinate_system_x",
                                                  "l_gaze_origin_in_user_coordinate_system_y",
                                                  "l_gaze_origin_in_user_coordinate_system_z",
                                                  "r_gaze_origin_in_user_coordinate_system_x",
                                                  "r_gaze_origin_in_user_coordinate_system_y",
                                                  "r_gaze_origin_in_user_coordinate_system_z"])

    df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(float) * opt["xres"]
    df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(float) * opt["yres"]
    df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(float) * opt["xres"]
    df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(float) * opt["yres"]
    df_eyetracking["l_valid"] = df_eyetracking["l_valid"].astype(int)
    df_eyetracking["r_valid"] = df_eyetracking["r_valid"].astype(int)

    df_eyetracking["l_miss_x"] = df_eyetracking.apply(lambda row: row["l_display_x"] < -opt["xres"] or row["l_display_x"] > 2 * opt["xres"], axis=1)
    df_eyetracking["l_miss_y"] = df_eyetracking.apply(lambda row: row["l_display_y"] < -opt["yres"] or row["l_display_y"] > 2 * opt["yres"], axis=1)
    df_eyetracking["r_miss_x"] = df_eyetracking.apply(lambda row: row["r_display_x"] < -opt["xres"] or row["r_display_x"] > 2 * opt["xres"], axis=1)
    df_eyetracking["r_miss_y"] = df_eyetracking.apply(lambda row: row["r_display_y"] < -opt["yres"] or row["r_display_y"] > 2 * opt["yres"], axis=1)

    df_eyetracking["l_miss"] = df_eyetracking.apply(lambda row: row["l_miss_x"] or row["l_miss_y"] or not row["l_valid"] >= 1, axis=1)
    df_eyetracking["r_miss"] = df_eyetracking.apply(lambda row: row["r_miss_x"] or row["r_miss_y"] or not row["r_valid"] >= 1, axis=1)

    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_x"] = opt["missingx"]
    df_eyetracking.loc[df_eyetracking["l_miss"], "l_display_y"] = opt["missingy"]
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_x"] = opt["missingx"]
    df_eyetracking.loc[df_eyetracking["r_miss"], "r_display_y"] = opt["missingy"]

    df_eyetracking = df_eyetracking.drop(columns=["l_miss_x", "l_miss_y", "r_miss_x", "r_miss_y", "l_miss", "r_miss"])

    df_eyetracking.rename(columns={"l_display_x": "L_X",
                                   "l_display_y": "L_Y",
                                   "r_display_x": "R_X",
                                   "r_display_y": "R_Y",
                                   "l_valid" : "LValidity",
                                   "r_valid" : "RValidity"}, inplace=True)

    # rename columns
    try:
        fix, data, par = I2MC.I2MC(df_eyetracking, opt, logging=False)
    except Exception as e:
        print('\t\tError in file {}: {}'.format(__file__, e))
        continue

    if not fix:
        print('{}-{}: Fixation calculation had some Problem'.format(row["Participant"], row["Algorithm"]))
        continue

    df_eyetracking["FixationNumber"] = np.nan
    df_eyetracking["FixationDuration"] = np.nan
    df_eyetracking["FixationX"] = np.nan
    df_eyetracking["FixationY"] = np.nan
    df_eyetracking["FixationFlankedDataLoss"] = np.nan
    df_eyetracking["FixationFracInterpreted"] = np.nan
    df_eyetracking["FixationRMSxy"] = np.nan
    df_eyetracking["FixationBCEA"] = np.nan
    df_eyetracking["FixationRangeX"] = np.nan
    df_eyetracking["FixationRangeY"] = np.nan

    display(fix)

1-IsPrime: Fixation Calculation


{'cutoff': 1.3492060479083539,
 'start': array([], dtype=int32),
 'end': array([], dtype=int32),
 'startT': array([], dtype=float64),
 'endT': array([], dtype=float64),
 'dur': array([], dtype=float64),
 'xpos': array([], dtype=float64),
 'ypos': array([], dtype=float64),
 'flankdataloss': array([], dtype=float64),
 'fracinterped': array([], dtype=float64),
 'RMSxy': array([], dtype=float64),
 'BCEA': array([], dtype=float64),
 'fixRangeX': array([], dtype=float64),
 'fixRangeY': array([], dtype=float64)}

1-SiebDesEratosthenes: Fixation Calculation


{'cutoff': 1.3441139990784154,
 'start': array([], dtype=int32),
 'end': array([], dtype=int32),
 'startT': array([], dtype=float64),
 'endT': array([], dtype=float64),
 'dur': array([], dtype=float64),
 'xpos': array([], dtype=float64),
 'ypos': array([], dtype=float64),
 'flankdataloss': array([], dtype=float64),
 'fracinterped': array([], dtype=float64),
 'RMSxy': array([], dtype=float64),
 'BCEA': array([], dtype=float64),
 'fixRangeX': array([], dtype=float64),
 'fixRangeY': array([], dtype=float64)}

1-IsAnagram: Fixation Calculation


...

{'cutoff': 1.3961821098430025,
 'start': array([], dtype=int32),
 'end': array([], dtype=int32),
 'startT': array([], dtype=float64),
 'endT': array([], dtype=float64),
 'dur': array([], dtype=float64),
 'xpos': array([], dtype=float64),
 'ypos': array([], dtype=float64),
 'flankdataloss': array([], dtype=float64),
 'fracinterped': array([], dtype=float64),
 'RMSxy': array([], dtype=float64),
 'BCEA': array([], dtype=float64),
 'fixRangeX': array([], dtype=float64),
 'fixRangeY': array([], dtype=float64)}

1-Power: Fixation Calculation


{'cutoff': 1.4464751308562585,
 'start': array([], dtype=int32),
 'end': array([], dtype=int32),
 'startT': array([], dtype=float64),
 'endT': array([], dtype=float64),
 'dur': array([], dtype=float64),
 'xpos': array([], dtype=float64),
 'ypos': array([], dtype=float64),
 'flankdataloss': array([], dtype=float64),
 'fracinterped': array([], dtype=float64),
 'RMSxy': array([], dtype=float64),
 'BCEA': array([], dtype=float64),
 'fixRangeX': array([], dtype=float64),
 'fixRangeY': array([], dtype=float64)}

1-BinarySearch: Fixation Calculation


KeyboardInterrupt: 